In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

from shared.constants import DatasetPath

In [3]:
DATASET = DatasetPath('ucidata-zachary')

In [4]:
spark = (SparkSession.builder
         .appName(f'{DATASET}_preprocess')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .getOrCreate())

22/01/03 23:32:55 WARN Utils: Your hostname, megatron resolves to a loopback address: 127.0.1.1; using 192.168.1.89 instead (on interface enp7s0)
22/01/03 23:32:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/03 23:32:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
schema = T.StructType([
    T.StructField('src', T.IntegerType(), False),
    T.StructField('dst', T.IntegerType(), False),
])

df = (
    spark.read.csv(DATASET.raw_str('out.ucidata-zachary'), sep=' ', header=False, schema=schema)
        .withColumn('index', F.monotonically_increasing_id())
        .filter(F.col('index') > 1)
        .withColumnRenamed('_c0', 'src')
        .withColumnRenamed('_c1', 'dst')
)
df.head(5)

[Row(src=1, dst=2, index=2),
 Row(src=1, dst=3, index=3),
 Row(src=2, dst=3, index=4),
 Row(src=1, dst=4, index=5),
 Row(src=2, dst=4, index=6)]

In [6]:
df_nodes = (
    df.select(F.col('src').alias('id'))
        .union(df.select(F.col('dst').alias('id')))
        .distinct()
)
df_nodes.head(5)

[Row(id=31), Row(id=28), Row(id=27), Row(id=26), Row(id=1)]

In [7]:
df_edges = (
    df.select(F.col('src'), F.col('dst'))
        .distinct()
)
df_edges.head(5)

[Row(src=24, dst=34),
 Row(src=19, dst=33),
 Row(src=24, dst=26),
 Row(src=1, dst=7),
 Row(src=2, dst=3)]

In [8]:
df_nodes.write.parquet(DATASET.processed_str('nodes_Pupil'), mode='overwrite')
df_edges.write.parquet(DATASET.processed_str('nodes_TRAINS'), mode='overwrite')

In [9]:
from datasets.build_schema import build_schema

build_schema(
    spark,
    name=str(DATASET),
    nodes=[
        ('Pupil', DATASET.processed_str('nodes_Pupil')),
    ],
    edges=[
        ('Trains', 'Pupil', 'Pupil', DATASET.processed_str('nodes_TRAINS')),
    ]
)

[2022-01-03 23:33:07,479][/dd_volume/Development/Python/Thesis/code/datasets/datasets/build_schema.py][DEBUG] Merging old schema for ucidata-zachary


DatasetSchema(name='ucidata-zachary', prefix='UcidataZachary', database='ucidata-zachary', description="Zachary's karate club dataset", nodes=[NodeSchema(label='Pupil', path='data/processed/ucidata-zachary/nodes_Pupil', properties=[Property(name='id', type='int', ignore=False, label=True)])], edges=[EdgeSchema(type='TRAINS', source='Pupil', target='Pupil', path='data/processed/ucidata-zachary/nodes_TRAINS', properties=[Property(name='src', type='int', ignore=False, label=False), Property(name='dst', type='int', ignore=False, label=False)])])